
# Backgammon Setup
<img src="https://media.istockphoto.com/id/184201541/vector/backgammon-board-illustration-with-dice.jpg?s=612x612&w=0&k=20&c=vsrDHXc3Ve_xvOf44SCtlLkjcTwjdRvAUkW-lNOwBNY=">

This is the initial configuration of the board of the backgammon game. The indexing of the board is done from bottom-right to top-right in anti-clockwise direction. Black moves anti-clockwise while while moves clockwise. I feel this is important to explicitly state as there are different variations of the game.


# Input

There are 26 lines of input. The first 24 lines indicate the number of coins held by player 1 or player 2 at that particular point (long triangle). The next two lines indicate the roll of the die.


# References

I have used the official github repository of the textbook shared by the IC as reference - https://github.com/aimacode/aima-python/blob/master/games.py


In [381]:
import copy
import time
import random

infinity = float('inf')
player = 0
MAX_DEPTH = 1 # will be changed while considering multiple TCs
dummy_action = [[0,0], [0,0]]

# If you roll the same number on both the die, the roll is doubled

'''
I wrote the following code to test for larger values of MAX_DEPTH
Instead of the 21 possibilities, I am randomnly considering 6
'''
list1 = [1, 2, 3, 4, 5, 6]
all_rolls = []
for i in range(6):
  die1 = random.choice(list1)
  die2 = random.choice(list1)
  all_rolls.append([die1,die2] if die1!=die2 else [die1]*4)


# all_rolls gets overriden here tho considering all 21 possibilites
all_rolls = []
for i in range(1, 7):
  for j in range(i, 7):
    all_rolls.append([i, j] if i != j else [i] * 4)

# print(len(all_rolls)) - should be 21

In [382]:
class State:
    def __init__(self, board, bar, player):
        self.board = board
        self.bar = bar
        self.player = player
        self.moves = []

    # if there is a coin on the bar, that has to be moved first with no exception
    def get_bar_move(self, board, roll, player):
        if player == 2:
          index = 25 - roll
        else:
          index = roll
        if len(board[index]) <= 1 or board[index][0] == player:
            return [-1, index]
        else:
            return []

    # bearing off is when a coin can be moved off the board
    # this function checks if it's possible       
    def can_bear_off(self,board, bar, player):
        bear_off = (bar[player] == 0)
        for i in range(1, 19):
            if player == 2:
              index = 25 - i
            else:
              index = i
            if bear_off:
                bear_off = (len(board[index]) == 0) or (board[index][0] != player)
        
        return bear_off

    # if bearing off is possible, this function generates the moves
    def get_bear_off_move(self, board, roll, player):
        poss_moves = []
        
        # this calculates direct moves
        if player == 1:
          index = 25 - roll
        else:
          index = roll
        
        if player == 2:
          end = 0
        else:
          end = 25
        
        '''
        I should have used extend instead of append 
        '''
        if len(board[index]) != 0 and board[index][0] == player:
            poss_moves.append([index, end])
            
        # this calculates inside home moves
        for i in range(1, 6):
            if player == 1:
              index = 18 + i
            else:
              index = 7 - i
      
            if len(board[index]) != 0 and board[index][0] == player:
                for j in range(1, 7 - i):
                    if player == 2:
                      index2 = index - j
                    else:
                      index2 = index + j
                    if j == roll:
                        if (len(board[index2]) == 0) or (len(board[index2]) == 1) or (board[index2][0] == player):
                            poss_moves.append([index, index2])
                    
        
        if len(poss_moves) != 0:
            return poss_moves
        else:
            for i in range(1, 7):
                if player == 1:
                  end = 25
                  index = 25 - i
                else:
                  end = 0
                  index = i
                if len(board[index]) != 0 and board[index][0] == player:
                    if i < roll:
                        return [[index, end]]
        return poss_moves

    # to generate all other moves               
    def get_normal_moves(self, board, roll, player):
        poss_moves = []
        for i in range(1, 24):
            if player == 2:
              index = 25 - i
            else:
              index = i
            if len(board[index]) != 0 and board[index][0] == player:
                for j in range(1, min(7, 25 - i)):
                    index2 = (index - j) if (player == 2) else (index + j)
                    if j == roll:
                        if len(board[index2]) <= 1 or board[index2][0] == player:
                            poss_moves.append([index, index2])
                    
        return poss_moves
    
    
    
    # this function generates all possible moves
    def get_moves(self, board, bar, dice_rolls, player, mv, moves):
        if len(dice_rolls)==0:
            moves.append(mv)
            return
        roll = dice_rolls[0]
        if bar[player] != 0:
            move = self.get_bar_move(board, roll, player)
            if len(move) != 0:
                bar[player] -= 1
                if len(board[move[1]]) == 1 and board[move[1]][0] != player:
                    board[move[1]].pop()
                board[move[1]].append(player)
                self.get_moves(board, bar, dice_rolls[1:], player, mv+[move], moves)
            else:
                self.get_moves(board, bar, dice_rolls[1:], player, mv, moves)
        else:
            if self.can_bear_off(board, bar, player):
                move = self.get_bear_off_move(board, roll, player)
            else:
                move = self.get_normal_moves(board, roll, player)
            if len(move) !=0:
                for m in move:
                    temp_board = copy.deepcopy(board)
                    temp_board[m[0]].pop()
                    if len(temp_board[m[1]]) == 1 and temp_board[m[1]][0] != player:
                        temp_board[m[1]].pop()
                    temp_board[m[1]].append(player)
                    self.get_moves(temp_board, bar, dice_rolls[1:], player, mv+[m], moves)
            else:
                self.get_moves(board, bar, dice_rolls[1:], player, mv, moves)

In [383]:
class Backgammon:
    def opponent(self, player):
        if player == 1:
          return 2
        return 1
    
    def actions(self, state, dice_rolls):
        board = copy.deepcopy(state.board)
        bar = copy.deepcopy(state.bar)
        player = state.player
        moves = []
        if dice_rolls is not None:
            if len(dice_rolls) == 4:
                state.get_moves(copy.deepcopy(board), copy.deepcopy(bar), copy.deepcopy(dice_rolls), player, [], moves)
            else:
                m1 = []
                m2 = []
                state.get_moves(copy.deepcopy(board), copy.deepcopy(bar), copy.deepcopy(dice_rolls), player, [], m1)
                state.get_moves(copy.deepcopy(board), copy.deepcopy(bar), copy.deepcopy(list(reversed(dice_rolls))), player, [], m2)
                moves = m1 + m2
        return moves
    
    def result(self, state, move=None):
        board = copy.deepcopy(state.board)
        bar = copy.deepcopy(state.bar)
        player = state.player
        if move is None:
            return State(board, bar, self.opponent(player))
        for m in move:
            if m[0] == -1:
                bar[player]-=1
                if len(board[m[1]]) == 1 and board[m[1]][0] != player:
                    bar[board[m[1]][0]] += 1
                    board[m[1]].pop()
                board[m[1]].append(player)
            else:
                board[m[0]].pop()
                if len(board[m[1]]) == 1 and board[m[1]][0] != player:
                    bar[board[m[1]][0]] += 1
                    board[m[1]].pop()
                
                board[m[1]].append(player)
        return State(board, bar, self.opponent(player))
    
    def utility(self, state, player):
        return None

# Evaluation Function

To show alpha-beta pruning is effective, it is absolutely essential for a very good evaluation function. I have, therefore, experimented with different functions and chose the one which performed the best.

In [384]:
# this is the evalution function that is called when depth reaches 0
# it rewards coins that are closer to the home board
# it penalises coins of the opponenet that are closer to the opponent's home board

def eval_fn(state):
    board = state.board
    m_player = player
    bar = state.bar
    v = 0
    distance = 0

    # penalising coins that are far away from the end
    # rewarding opp coins that are far from its end
    for i in range(1, 25):
        if len(board[i]) > 1 and board[i][0] == m_player:
            if m_player == 1:
              distance += board[i][1] * (25 - i)
            else:
              distance += board[i][1] * i
        if len(board[i]) > 1 and board[i][0] != m_player:
            if m_player == 1:
              distance -= board[i][1] * i
            else:
              distance -= board[i][1] * (25 - i)

    v = -50*distance

    # rewarding coins that are on the home ground
    home_coins = 0
    for i in range(1, 6):
        index = 18 + i if (1 == m_player) else (7 - i)
        if len(board[index]) > 0 and board[index] == m_player:
            home_coins += len(board[index])
    v += 25*home_coins

    # rewarding coins that have gone past the home ground
    v += 100*(len(board[25]) if m_player==1 else len(board[0]))

    # penalising coins that are on the bar
    v -= 75*bar[m_player]

    # penalising opp coins that are on its home ground
    opp_coins = 0
    for i in range(1, 6):
        index = 18 + i if (2 == m_player) else (7 - i)
        if len(board[index])>0 and board[index] != m_player:
            opp_coins += len(board[index])
    v -= 100*opp_coins
    return v

In [385]:
'''

Just experimenting with a different evaluation function

def pipCount(state, is_player1):
  board = state.board
  m_player = player
  bar = state.bar

  sum = 0
  for i in range(1, 25):
    if is_player1 and len(board[i])>0 and board[i][0] == 1:
      sum += len(board[i]) * (24 - i)
    elif is_player1 == False and len(board[i])>0 and board[i][0] == 2:
      sum += len(board[i]) * (i + 1)
		
  if is_player1:
    sum += -10 * bar[0]
    sum += 10 * bar[1]
  else:
    sum += -10 * bar[1]
    sum += 10 * bar[0]
  return sum

def isSafe(state, index, is_player1):
  index += 1
  board = state.board
  if is_player1:
    return len(board[index]) > 0 and board[index] == 1
  else:
    return len(board[index]) > 0 and board[index] == 2
		
def scoreForGoodCoverage(state, is_player1):
  score = 0;
  if is_player1:
    for i in range(0, 17): 
      if isSafe(state, i, is_player1):
        score += 1
  else:
    for i in range(23, 6, -1):
      if isSafe(state, i, is_player1): 
        score += 1
  return score
	

def eval_fn(state):
  pipDifference = pipCount(state, False) - pipCount(state, True);
  coverageDifference = scoreForGoodCoverage(state, True) - scoreForGoodCoverage(state, False)
  #unsafeDifference = scoreForUnsafePieces(state, false) - scoreForUnsafePieces(state, true)
  score = 100*pipDifference + 50*coverageDifference #+ 50*unsafeDifference
  if player == 1:
    return score
  return -1*score

'''

In [386]:
def max_value(state, game, roll, depth):
    actions = game.actions(state, roll)
    v = -infinity
    if not actions:
        return expectinode(game.result(state), game, depth)
    for a in actions:
        v = max(v, expectinode(game.result(state, a), game, depth))
    return v

def min_value(state, game, roll, depth):
    actions = game.actions(state, roll)
    v = infinity
    depth -= 1
    if not actions:
        return expectinode(game.result(state), game, depth)
    for a in actions:
        v = min(v, expectinode(game.result(state, a), game, depth))
    return v

def expectinode(state, game, depth):
    if depth == 0:
        return eval_fn(state)
    if player == state.player:
        func = max_value
    else: 
        func = min_value
    v = 0
    for roll in all_rolls:
        v += (1/36 if len(roll) == 4 else 1/18) * func(state, game, roll, depth)
    return v


def get_best_move(state, game, dice_rolls):
    best_score = -infinity
    best_action = None

    depth = MAX_DEPTH
    actions = game.actions(state, dice_rolls)
    #print("\n")
    print("Action\t\t\tScore")
    for a in actions:
        v = expectinode(game.result(state,a), game, depth)
        print(a, "\t", v)
        if v > best_score:
            best_score = v
            best_action = a
    print("\n")
    return best_action

In [387]:
def max_value_alphabeta(state, game, roll, alpha, beta, depth):
    actions = game.actions(state, roll)
    
    v = -infinity
    
    if not actions:
        return expectinode_alphabeta(game.result(state), game, alpha, beta, depth)
    for a in actions:
        v = max(v, expectinode_alphabeta(game.result(state, a), game, alpha, beta, depth))
        if v >= beta: # magic of alpha-beta pruning happens here
            return v
        alpha = max(alpha, v)
    return v
       
def min_value_alphabeta(state, game, roll, alpha, beta, depth):
    actions = game.actions(state, roll)
    
    v = infinity
    depth -= 1
    if not actions:
        return expectinode_alphabeta(game.result(state), game, alpha, beta, depth)
    for a in actions:
        v = min(v, expectinode_alphabeta(game.result(state, a), game, alpha, beta, depth))
        if v <= alpha: # magic of alpha-beta pruning happens here
            return v
        beta = min(beta, v)
    return v

def expectinode_alphabeta(state, game, alpha, beta, depth):
    if depth == 0:
        return eval_fn(state)
    if player == state.player:
        func = max_value_alphabeta
    else:
        func = min_value_alphabeta
    v = 0
    for roll in all_rolls:
        v += (1/36 if len(roll) == 4 else 1/18) * func(state, game, roll, alpha, beta, depth)
    return v

def get_best_move_alphabeta(state, game, dice_rolls):
    best_score = -infinity
    best_action = None

    alpha = -infinity
    beta = infinity

    depth = MAX_DEPTH
    actions = game.actions(state, dice_rolls)
    
    print("Action\t\t\tScore")
    for a in actions:
        v = expectinode_alphabeta(game.result(state,a), game, alpha, beta, depth)
        print(a, "\t", v)
        if v > best_score:
            best_score = v
            best_action = a
    print("\n")
    return best_action

In [388]:
# assuming it is player 1's turn
player = 1

def startGame(board, bar, dice_rolls):
  board.append([])
  for i in range(24):
    x = list(map(int, input().strip().split()))
    if len(x) == 2:
      board.append([x[1]] * x[0])
    else:
      board.append([])
  board.append([])

  bar.append(0)
  bar.append(0)
  bar.append(0)

  #print(board)
  die1 = int(input())
  die2 = int(input())

  dice_rolls.append(die1)
  dice_rolls.append(die2)

  # if the same number appears on both the die
  if(die1 == die2):
    dice_rolls.append(die1)
    dice_rolls.append(die2)

  dice_rolls.sort()
  game_state = State(board, bar, player)
  game = Backgammon()

  #print(game.actions(game_state, dice_rolls))
  print("Best move with alpha-beta pruning: ")
  start_time = time.time()
  moves = get_best_move_alphabeta(game_state, game, dice_rolls)
  if moves is not None:
    print("\n".join(" ".join(str(m) for m in move) for move in moves))
  tot_time = time.time() - start_time
  print("It takes", tot_time, "seconds")

  print("----------------------------------------------------------\n")
  
  print("Best move without alpha-beta pruning: ")
  start_time = time.time()
  moves = get_best_move(game_state, game, dice_rolls)
  if moves is not None:
    print("\n".join(" ".join(str(m) for m in move) for move in moves))
  tot_time = time.time() - start_time
  print("It takes", tot_time, "seconds\n")




# Example 1
In this example, I have chosen a configuration where player-1 is overwhelmingly winning. However, there is barely any difference in runtime due to the fact there are very few combinations of moves to be considered for player-1

## MAX_DEPTH = 1

In [389]:
board = []
bar = []
dice_rolls = []

# This is a popular way to take input in Competitive Programming. I have decided to use it since it's a multiline input.

S = """
2 2
0
0
0
0
0
0
0
0
0
0
5 2
0
0
0
0
3 2
5 2
0
0
3 1
5 1
5 1
2 1
6
5
""".strip("\n").split("\n")
S.reverse()
input = S.pop

startGame(board, bar, dice_rolls)


Best move with alpha-beta pruning: 
Action			Score
[[24, 25], [24, 25]] 	 2836.111111111111
[[24, 25], [24, 25]] 	 2836.111111111111


24 25
24 25
It takes 0.56646728515625 seconds
----------------------------------------------------------

Best move without alpha-beta pruning: 
Action			Score
[[24, 25], [24, 25]] 	 2836.111111111111
[[24, 25], [24, 25]] 	 2836.111111111111


24 25
24 25
It takes 0.858508825302124 seconds



# Example 2

In this example, I have chosen a more neutral configuration. Alpha-beta pruning slights reduces the runtime.

## MAX_DEPTH = 1

In [378]:
board = []
bar = []
dice_rolls = []
S = """
2 2
0
5 1
0
0
0
0
0
0
0
0
5 2
0
2 1
0
0
3 2
5 2
0
0
3 1
0
5 1
0
2
1
""".strip("\n").split("\n")
S.reverse()
input = S.pop


startGame(board, bar, dice_rolls)

Best move with alpha-beta pruning: 
Action			Score
[[3, 4], [3, 5]] 	 1134.7222222222222
[[3, 4], [4, 6]] 	 1132.638888888889
[[3, 4], [14, 16]] 	 1138.888888888889
[[3, 4], [21, 23]] 	 1131.9444444444446
[[14, 15], [3, 5]] 	 1138.1944444444443
[[14, 15], [14, 16]] 	 1136.8055555555557
[[14, 15], [21, 23]] 	 1136.8055555555557
[[21, 22], [3, 5]] 	 1133.3333333333333
[[21, 22], [14, 16]] 	 1138.888888888889
[[21, 22], [21, 23]] 	 1330.5555555555552
[[21, 22], [22, 24]] 	 1130.5555555555557
[[23, 24], [3, 5]] 	 1133.3333333333333
[[23, 24], [14, 16]] 	 1138.888888888889
[[23, 24], [21, 23]] 	 1130.5555555555557
[[3, 5], [3, 4]] 	 1134.7222222222222
[[3, 5], [5, 6]] 	 1132.638888888889
[[3, 5], [14, 15]] 	 1138.1944444444443
[[3, 5], [21, 22]] 	 1133.3333333333333
[[3, 5], [23, 24]] 	 1133.3333333333333
[[14, 16], [3, 4]] 	 1138.888888888889
[[14, 16], [14, 15]] 	 1136.8055555555557
[[14, 16], [21, 22]] 	 1138.888888888889
[[14, 16], [23, 24]] 	 1138.888888888889
[[21, 23], [3, 4]] 	 1131

# EXAMPLE 3

Effectiveness of alpha-beta pruning is evident in this example. I have chosen the same configuration as Example-2. However, this time I won't call the evaluation function until 2 turns are completed for each player. Since, it was taking forever for this code to finish due to the large branching factor, I have randomnly chosen 3 die combinations which will be considered. This reduce the branching factor from 21 to 3. 

Note: This is purely to reduce the runtime and makes testing easier.

## MAX_DEPTH = 2

In [379]:
board = []
bar = []
dice_rolls = []
MAX_DEPTH = 2

all_rolls = []
for i in range(3):
  die1 = random.choice(list1)
  die2 = random.choice(list1)
  all_rolls.append([die1,die2] if die1!=die2 else [die1]*4)


S = """
2 2
0
5 1
0
0
0
0
0
0
0
0
5 2
0
2 1
0
0
3 2
5 2
0
0
3 1
0
5 1
0
2
1
""".strip("\n").split("\n")
S.reverse()
input = S.pop


startGame(board, bar, dice_rolls)

Best move with alpha-beta pruning: 
Action			Score
[[3, 4], [3, 5]] 	 4.86539780521262
[[3, 4], [4, 6]] 	 2.2890946502057608
[[3, 4], [14, 16]] 	 4.1323731138545945
[[3, 4], [21, 23]] 	 2.92352537722908
[[14, 15], [3, 5]] 	 4.711076817558299
[[14, 15], [14, 16]] 	 4.62534293552812
[[14, 15], [21, 23]] 	 4.70250342935528
[[21, 22], [3, 5]] 	 3.047839506172839
[[21, 22], [14, 16]] 	 4.629629629629629
[[21, 22], [21, 23]] 	 3.806584362139917
[[21, 22], [22, 24]] 	 2.8806584362139915
[[23, 24], [3, 5]] 	 3.047839506172839
[[23, 24], [14, 16]] 	 4.629629629629629
[[23, 24], [21, 23]] 	 2.8806584362139915
[[3, 5], [3, 4]] 	 4.86539780521262
[[3, 5], [5, 6]] 	 2.2890946502057608
[[3, 5], [14, 15]] 	 4.711076817558299
[[3, 5], [21, 22]] 	 3.047839506172839
[[3, 5], [23, 24]] 	 3.047839506172839
[[14, 16], [3, 4]] 	 4.1323731138545945
[[14, 16], [14, 15]] 	 4.62534293552812
[[14, 16], [21, 22]] 	 4.629629629629629
[[14, 16], [23, 24]] 	 4.629629629629629
[[21, 23], [3, 4]] 	 2.92352537722908
[[

# Conclusion
Clearly the best move calculated with alpha-beta pruning takes almost 90% less time to run than without alpha-beta pruning, with the same best moves suggestion. This shows how effective alpha-beta pruning is.